In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 !/.kaggle/kaggle.json

chmod: cannot access '!/.kaggle/kaggle.json': No such file or directory


In [3]:
import os
import kaggle

os.environ['KAGGLE_CONFIG_DIR'] = '/root/.kaggle/'
kaggle.api.authenticate()

kaggle.api.dataset_download_files('linkanjarad/body-parts-dataset', unzip=True)

# Body Parts Recognition Model

- Dataset from Kaggle [Link](https://www.kaggle.com/datasets/linkanjarad/body-parts-dataset).

In [4]:
import os
import random
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

## A function which build 2 seprate sets i.e., Training and testing sets.

In [5]:
def buildDataset(name):
    names = []
    sourcePath = f'/content/Body Parts Dataset/{name}'

    # Check if the source directory exists
    if not os.path.exists(sourcePath):
        print(f"Source directory '{sourcePath}' does not exist.")
        return

    # Create the destination directories if they don't exist
    trainPath = f'/content/dataset/train/{name}'
    testPath = f'/content/dataset/test/{name}'

    os.makedirs(trainPath, exist_ok=True)
    os.makedirs(testPath, exist_ok=True)

    for filename in os.listdir(sourcePath):
        names.append(filename)

    # Shuffling of the whole dataset
    random.shuffle(names)

    # Split dataset into train and test set
    train = names[0: int(0.8*len(names))]
    test = names[int(0.8*len(names)): len(names)+1]

    # Store the prepared data into separate dataset folder
    # Storing the training dataset
    for i in train:
        filePath = os.path.join(sourcePath, i)
        img = cv2.imread(filePath)
        cv2.imwrite(os.path.join(trainPath, i), img)

    # Storing the testing dataset
    for i in test:
        filePath = os.path.join(sourcePath, i)
        img = cv2.imread(filePath)
        cv2.imwrite(os.path.join(testPath, i), img)

In [6]:
dataset = ['Belly', 'Ear', 'Elbow', 'Eye', 'Foot', 'Hand', 'Knee', 'Neck', 'Nose', 'Shoulders']

# Building the train and test dataset for model training.
for name in dataset:
  buildDataset(name)

In [7]:
# path of the train and test dataset
trainPath = '/content/dataset/train'
testPath = '/content/dataset/test'

## Refactoring the images and scale them to fit for the model.

In [8]:
generator = ImageDataGenerator(rescale = 1.0/255.0)

# Normalise the train dataset
trainImageGenerator = generator.flow_from_directory(trainPath,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='categorical')

# Normalise the test dataset
testImageGenerator = generator.flow_from_directory(testPath,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='categorical')

Found 1669 images belonging to 10 classes.
Found 423 images belonging to 10 classes.


In [9]:
classMap = dict([v, k] for k, v in trainImageGenerator.class_indices.items())
print(classMap)

{0: 'Belly', 1: 'Ear', 2: 'Elbow', 3: 'Eye', 4: 'Foot', 5: 'Hand', 6: 'Knee', 7: 'Neck', 8: 'Nose', 9: 'Shoulders'}


## Building the model.
- Start with Convolution layers.
- Then Maxpooling layers.
- After a neural network, Creating a Fully-connected layers.
- Now, flatten all the layers in a single dimenstion to create a Fully-connected layers.
- There is a Dropout layer in between the Fully-connected layers.

In [10]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=[150, 150, 3]))
model.add(MaxPooling2D(2,))
model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(2))

# Flatten the feature map
model.add(Flatten())

# Add the fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# print(the model summary)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 75, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 37, 37, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 87616)             0         
                                                                 
 dense (Dense)               (None, 128)               1

### Compiling the model on **Adam** Optimizer with **Categorical Crossentropy** loss function.

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics='accuracy')

### Model training begins

In [12]:
model.fit(trainImageGenerator, epochs=15, verbose=1)

Epoch 1/15
53/53 [==============================] - 10s 64ms/step - loss: 2.6138 - accuracy: 0.1246
Epoch 2/15
53/53 [==============================] - 5s 93ms/step - loss: 2.1944 - accuracy: 0.2157
Epoch 3/15
53/53 [==============================] - 3s 60ms/step - loss: 1.9262 - accuracy: 0.3355
Epoch 4/15
53/53 [==============================] - 2s 32ms/step - loss: 1.5440 - accuracy: 0.5087
Epoch 5/15
53/53 [==============================] - 2s 34ms/step - loss: 1.1049 - accuracy: 0.6411
Epoch 6/15
53/53 [==============================] - 2s 44ms/step - loss: 0.7519 - accuracy: 0.7591
Epoch 7/15
53/53 [==============================] - 3s 50ms/step - loss: 0.4958 - accuracy: 0.8466
Epoch 8/15
53/53 [==============================] - 2s 32ms/step - loss: 0.3332 - accuracy: 0.9041
Epoch 9/15
53/53 [==============================] - 2s 32ms/step - loss: 0.2537 - accuracy: 0.9209
Epoch 10/15
53/53 [==============================] - 2s 32ms/step - loss: 0.2069 - accuracy: 0.9437
Epoch 11

In [13]:
history = model.evaluate(trainImageGenerator)

53/53 [==============================] - 2s 29ms/step - loss: 0.0137 - accuracy: 0.9964


In [14]:
print('Accuracy: {:.2f}%'.format(history[1] * 100))
print('Loss: {:.2f}%'.format(history[0] * 100))

Accuracy: 99.64%
Loss: 1.37%


### Model Ready with 99.64% Accuracy.